# Modeling [10 Marks]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp -r "/content/drive/MyDrive/IITG_assignments/internship/Dataset" "/content/internship"

In [4]:
!apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [5]:
!tree /content/internship/

/content/internship/
├── breathing_dataset.csv
├── combined_breathing_dataset.csv
├── combined_sleep_stage_dataset.csv
├── Dataset
│   ├── breathing_dataset.csv
│   ├── combined_breathing_dataset.csv
│   ├── combined_sleep_stage_dataset.csv
│   └── sleep_stage_dataset.csv
└── sleep_stage_dataset.csv

1 directory, 8 files


In [6]:
import os
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# Load datasets
breathing_df = pd.read_csv("/content/internship/combined_breathing_dataset.csv")
sleep_stage_df = pd.read_csv("/content/internship/combined_sleep_stage_dataset.csv")

# Display first 5 rows of each
print("Breathing Dataset (breathing_dataset.csv) — head():\n")
print(breathing_df.head(), "\n")

print("Sleep Stage Dataset (sleep_stage_dataset.csv) — head():\n")
print(sleep_stage_df.head())


Breathing Dataset (breathing_dataset.csv) — head():

  subject         window_start           window_end  flow_mean  thoracic_mean  \
0    AP01  2024-05-30 20:59:00  2024-05-30 20:59:30  -2.402243       0.951803   
1    AP01  2024-05-30 20:59:15  2024-05-30 20:59:45   0.979846       1.261949   
2    AP01  2024-05-30 20:59:30  2024-05-30 21:00:00  -2.549896       1.604093   
3    AP01  2024-05-30 20:59:45  2024-05-30 21:00:15  -3.223234       1.264346   
4    AP01  2024-05-30 21:00:00  2024-05-30 21:00:30  -2.825299       0.222740   

    label sleep_stage  overlap_seconds  
0  Normal        Wake              0.0  
1  Normal        Wake              0.0  
2  Normal        Wake              0.0  
3  Normal        Wake              0.0  
4  Normal        Wake              0.0   

Sleep Stage Dataset (sleep_stage_dataset.csv) — head():

  subject         window_start           window_end sleep_stage
0    AP01  2024-05-30 20:59:00  2024-05-30 20:59:30        Wake
1    AP01  2024-05-30 20:59

#1D CNN model

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, TimeDistributed
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [9]:
# Load datasets
breathing_df = pd.read_csv('/content/drive/MyDrive/IITG_assignments/internship/Dataset/combined_breathing_dataset.csv')
sleep_df = pd.read_csv('/content/drive/MyDrive/IITG_assignments/internship/Dataset/combined_sleep_stage_dataset.csv')

# Convert datetime columns
breathing_df['window_start'] = pd.to_datetime(breathing_df['window_start'])
breathing_df['window_end'] = pd.to_datetime(breathing_df['window_end'])
sleep_df['window_start'] = pd.to_datetime(sleep_df['window_start'])
sleep_df['window_end'] = pd.to_datetime(sleep_df['window_end'])

# Merge datasets - keep only the sleep_stage from sleep_df
merged_df = pd.merge(
    breathing_df.drop('sleep_stage', axis=1),  # Remove sleep_stage from breathing_df to avoid duplicate columns
    sleep_df[['subject', 'window_start', 'window_end', 'sleep_stage']],
    on=['subject', 'window_start', 'window_end'],
    how='inner'
)

# Verify merge
print("Merged dataset columns:", merged_df.columns)
print("\nMerged dataset head:")
print(merged_df.head())


Merged dataset columns: Index(['subject', 'window_start', 'window_end', 'flow_mean', 'thoracic_mean',
       'label', 'overlap_seconds', 'sleep_stage'],
      dtype='object')

Merged dataset head:
  subject        window_start          window_end  flow_mean  thoracic_mean  \
0    AP01 2024-05-30 20:59:00 2024-05-30 20:59:30  -2.402243       0.951803   
1    AP01 2024-05-30 20:59:15 2024-05-30 20:59:45   0.979846       1.261949   
2    AP01 2024-05-30 20:59:30 2024-05-30 21:00:00  -2.549896       1.604093   
3    AP01 2024-05-30 20:59:45 2024-05-30 21:00:15  -3.223234       1.264346   
4    AP01 2024-05-30 21:00:00 2024-05-30 21:00:30  -2.825299       0.222740   

    label  overlap_seconds sleep_stage  
0  Normal              0.0        Wake  
1  Normal              0.0        Wake  
2  Normal              0.0        Wake  
3  Normal              0.0        Wake  
4  Normal              0.0        Wake  


In [10]:
# Encode labels
label_encoder = LabelEncoder()
merged_df['label_encoded'] = label_encoder.fit_transform(merged_df['label'])

# One-hot encode sleep stage
sleep_stage_dummies = pd.get_dummies(merged_df['sleep_stage'], prefix='sleep')

# Features and labels
numeric_features = ['flow_mean', 'thoracic_mean']
X_numeric = merged_df[numeric_features]
X = pd.concat([X_numeric, sleep_stage_dummies], axis=1)
y = merged_df['label_encoded']
groups = merged_df['subject'].values

# Standardize numerical features
scaler = StandardScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Convert to numpy arrays
X = X.values.astype('float32')
y = y.values
groups = merged_df['subject'].values

# Reshape data for 1D CNN and Conv-LSTM (samples, timesteps, features)
X_reshaped = X.reshape(X.shape[0], 1, X.shape[1])

# Define Leave-One-Subject-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize dictionaries to store results
results_cnn = {'accuracy': [], 'precision': [], 'recall': [], 'sensitivity': [], 'specificity': []}
results_conv_lstm = {'accuracy': [], 'precision': [], 'recall': [], 'sensitivity': [], 'specificity': []}

In [11]:
# Define model creation functions
def create_1d_cnn(input_shape, num_classes):
    model = Sequential([
        Conv1D(64, kernel_size=1, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=1),
        Conv1D(128, kernel_size=1, activation='relu'),
        MaxPooling1D(pool_size=1),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

def calculate_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    num_classes = cm.shape[0]

    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'sensitivity': 0,
        'specificity': 0
    }

    # For binary classification
    if num_classes == 2:
        tn, fp, fn, tp = cm.ravel()
        metrics['sensitivity'] = tp / (tp + fn)
        metrics['specificity'] = tn / (tn + fp)
    # For multi-class classification
    else:
        # Calculate sensitivity (recall for each class)
        recalls = recall_score(y_true, y_pred, average=None, zero_division=0)
        metrics['sensitivity'] = np.mean(recalls)

        # Calculate specificity (requires more complex calculation)
        specificities = []
        for i in range(num_classes):
            tn = np.sum(np.delete(np.delete(cm, i, axis=0), i, axis=1))
            fp = np.sum(cm[:, i]) - cm[i, i]
            specificities.append(tn / (tn + fp))
        metrics['specificity'] = np.mean(specificities)

    return metrics, cm

In [12]:
# Perform Leave-One-Subject-Out cross-validation
for fold, (train_idx, test_idx) in enumerate(logo.split(X, y, groups)):
    print(f"\n=== Fold {fold+1} - Testing on subject {groups[test_idx[0]]} ===")

    # Split data
    X_train, X_test = X_reshaped[train_idx], X_reshaped[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Get input shape
    input_shape = (X_train.shape[1], X_train.shape[2])
    num_classes = len(np.unique(y))

    # Train and evaluate 1D CNN
    print("\nTraining 1D CNN...")
    cnn_model = create_1d_cnn(input_shape, num_classes)
    cnn_history = cnn_model.fit(X_train, y_train,
                               epochs=20,
                               batch_size=32,
                               validation_split=0.2,
                               verbose=1)

    # Evaluate 1D CNN
    cnn_pred = np.argmax(cnn_model.predict(X_test), axis=1)

    # Calculate metrics
    cnn_metrics, cnn_cm = calculate_metrics(y_test, cnn_pred)

    # Store results
    results_cnn['accuracy'].append(cnn_metrics['accuracy'])
    results_cnn['precision'].append(cnn_metrics['precision'])
    results_cnn['recall'].append(cnn_metrics['recall'])
    results_cnn['sensitivity'].append(cnn_metrics['sensitivity'])
    results_cnn['specificity'].append(cnn_metrics['specificity'])

    # Print metrics
    print(f"\n1D CNN Metrics for fold {fold+1}:")
    print(f"Accuracy: {cnn_metrics['accuracy']:.4f}")
    print(f"Precision: {cnn_metrics['precision']:.4f}")
    print(f"Recall: {cnn_metrics['recall']:.4f}")
    print(f"Sensitivity: {cnn_metrics['sensitivity']:.4f}")
    print(f"Specificity: {cnn_metrics['specificity']:.4f}")
    print("Confusion Matrix:")
    print(cnn_cm)



=== Fold 1 - Testing on subject AP01 ===

Training 1D CNN...
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8734 - loss: 0.5876 - val_accuracy: 0.7978 - val_loss: 1.3863
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9484 - loss: 0.1963 - val_accuracy: 0.7978 - val_loss: 1.2241
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9470 - loss: 0.2022 - val_accuracy: 0.7978 - val_loss: 1.1548
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9439 - loss: 0.2079 - val_accuracy: 0.7978 - val_loss: 1.1096
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9404 - loss: 0.2151 - val_accuracy: 0.7978 - val_loss: 1.2742
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9516 - loss: 0.1823 - val_accuracy: 0.7978 - val_loss: 1.2759
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9458 - loss: 0.1954 - val_accuracy: 0.7978 - val_loss: 1.2669
Epoch 8/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9445 - loss: 0.2054 - val_accuracy: 0.7978 - val_

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8951 - loss: 0.5066 - val_accuracy: 0.7987 - val_loss: 1.1513
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9541 - loss: 0.1888 - val_accuracy: 0.7987 - val_loss: 1.0620
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9553 - loss: 0.1711 - val_accuracy: 0.7987 - val_loss: 0.9823
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9514 - loss: 0.1797 - val_accuracy: 0.7987 - val_loss: 0.8886
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9644 - loss: 0.1512 - val_accuracy: 0.7987 - val_loss: 0.9370
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9567 - loss: 0.1690 - val_accuracy: 0.7987 - val_loss: 0.8669
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9533 - loss: 0.1734 - val_accuracy: 0.7987 - val_loss: 0.8656
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9567 - loss: 0.1698 - val_accuracy: 0.7987 - val_

...

178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8433 - loss: 0.6184 - val_accuracy: 0.8000 - val_loss: 1.0199
Epoch 2/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9361 - loss: 0.2396 - val_accuracy: 0.8000 - val_loss: 0.8646
Epoch 3/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9366 - loss: 0.2378 - val_accuracy: 0.8000 - val_loss: 0.8621
Epoch 4/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9312 - loss: 0.2435 - val_accuracy: 0.8000 - val_loss: 1.0081
Epoch 5/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9376 - loss: 0.2354 - val_accuracy: 0.8000 - val_loss: 0.8841
Epoch 6/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9323 - loss: 0.2341 - val_accuracy: 0.8000 - val_loss: 0.8011
Epoch 7/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9349 - loss: 0.2364 - val_accuracy: 0.8000 - val_loss: 0.9626
Epoch 8/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9326 - loss: 0.2365 - val_accuracy: 0.8000 - val_

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9077 - loss: 0.5673 - val_accuracy: 0.7968 - val_loss: 0.9001
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9576 - loss: 0.1806 - val_accuracy: 0.7968 - val_loss: 0.9612
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9502 - loss: 0.1905 - val_accuracy: 0.7968 - val_loss: 0.7960
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9511 - loss: 0.1863 - val_accuracy: 0.7968 - val_loss: 0.7063
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9600 - loss: 0.1571 - val_accuracy: 0.7968 - val_loss: 0.6953
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9491 - loss: 0.1907 - val_accuracy: 0.7968 - val_loss: 0.7649
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9540 - loss: 0.1876 - val_accuracy: 0.7968 - val_loss: 0.7806
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9562 - loss: 0.1704 - val_accuracy: 0.7968 - val_

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9051 - loss: 0.4978 - val_accuracy: 0.8988 - val_loss: 0.3355
Epoch 2/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9546 - loss: 0.1921 - val_accuracy: 0.8988 - val_loss: 0.3342
Epoch 3/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9570 - loss: 0.1704 - val_accuracy: 0.8988 - val_loss: 0.3337
Epoch 4/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9587 - loss: 0.1677 - val_accuracy: 0.8988 - val_loss: 0.3370
Epoch 5/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9635 - loss: 0.1540 - val_accuracy: 0.8988 - val_loss: 0.3326
Epoch 6/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9586 - loss: 0.1637 - val_accuracy: 0.8988 - val_loss: 0.3481
Epoch 7/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9570 - loss: 0.1697 - val_accuracy: 0.8988 - val_loss: 0.3647
Epoch 8/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9602 - loss: 0.1574 - val_accuracy: 0.8988 - val_

In [13]:

# Calculate and print aggregated results
def print_aggregated_results(results, model_name):
    print(f"\n=== Aggregated Results for {model_name} ===")
    for metric in results:
        values = results[metric]
        print(f"{metric.capitalize()}: Mean = {np.mean(values):.4f}, Std = {np.std(values):.4f}")

print_aggregated_results(results_cnn, "1D CNN")



=== Aggregated Results for 1D CNN ===
Accuracy: Mean = 0.9212, Std = 0.0575
Precision: Mean = 0.8519, Std = 0.1036
Recall: Mean = 0.9212, Std = 0.0575
Sensitivity: Mean = 0.2900, Std = 0.0554
Specificity: Mean = 0.7100, Std = 0.0554


#conv LSTM model

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, TimeDistributed, Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt

In [15]:
# Load datasets
breathing_df = pd.read_csv('/content/drive/MyDrive/IITG_assignments/internship/Dataset/combined_breathing_dataset.csv')
sleep_df = pd.read_csv('/content/drive/MyDrive/IITG_assignments/internship/Dataset/combined_sleep_stage_dataset.csv')

# Convert datetime columns
breathing_df['window_start'] = pd.to_datetime(breathing_df['window_start'])
breathing_df['window_end'] = pd.to_datetime(breathing_df['window_end'])
sleep_df['window_start'] = pd.to_datetime(sleep_df['window_start'])
sleep_df['window_end'] = pd.to_datetime(sleep_df['window_end'])

# Merge datasets - keep only the sleep_stage from sleep_df
merged_df = pd.merge(
    breathing_df.drop('sleep_stage', axis=1),  # Remove sleep_stage from breathing_df to avoid duplicate columns
    sleep_df[['subject', 'window_start', 'window_end', 'sleep_stage']],
    on=['subject', 'window_start', 'window_end'],
    how='inner'
)

# Verify merge
print("Merged dataset columns:", merged_df.columns)
print("\nMerged dataset head:")
print(merged_df.head())

Merged dataset columns: Index(['subject', 'window_start', 'window_end', 'flow_mean', 'thoracic_mean',
       'label', 'overlap_seconds', 'sleep_stage'],
      dtype='object')

Merged dataset head:
  subject        window_start          window_end  flow_mean  thoracic_mean  \
0    AP01 2024-05-30 20:59:00 2024-05-30 20:59:30  -2.402243       0.951803   
1    AP01 2024-05-30 20:59:15 2024-05-30 20:59:45   0.979846       1.261949   
2    AP01 2024-05-30 20:59:30 2024-05-30 21:00:00  -2.549896       1.604093   
3    AP01 2024-05-30 20:59:45 2024-05-30 21:00:15  -3.223234       1.264346   
4    AP01 2024-05-30 21:00:00 2024-05-30 21:00:30  -2.825299       0.222740   

    label  overlap_seconds sleep_stage  
0  Normal              0.0        Wake  
1  Normal              0.0        Wake  
2  Normal              0.0        Wake  
3  Normal              0.0        Wake  
4  Normal              0.0        Wake  


# Preprocessing

In [16]:
# Encode labels
label_encoder = LabelEncoder()
merged_df['label_encoded'] = label_encoder.fit_transform(merged_df['label'])

# One-hot encode sleep stage
sleep_stage_dummies = pd.get_dummies(merged_df['sleep_stage'], prefix='sleep')

# Features and labels
numeric_features = ['flow_mean', 'thoracic_mean']
X_numeric = merged_df[numeric_features]
X = pd.concat([X_numeric, sleep_stage_dummies], axis=1)
y = merged_df['label_encoded']
groups = merged_df['subject'].values

# Standardize numerical features
scaler = StandardScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# Convert to numpy arrays
X = X.values.astype('float32')
y = y.values
groups = merged_df['subject'].values

# Reshape data for Conv-LSTM (samples, timesteps, features, channels)
X_reshaped = X.reshape(X.shape[0], 1, X.shape[1], 1)  # (samples, timesteps, features, channels)

# Define Leave-One-Subject-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize dictionaries to store results
results_conv_lstm = {'accuracy': [], 'precision': [], 'recall': [], 'sensitivity': [], 'specificity': []}


In [17]:
def create_conv_lstm(input_shape, num_classes):
    model = Sequential([
        # Input shape: (timesteps, features, channels)
        TimeDistributed(Conv1D(64, kernel_size=1, activation='relu'), input_shape=input_shape),
        TimeDistributed(MaxPooling1D(pool_size=1)),
        TimeDistributed(Conv1D(128, kernel_size=1, activation='relu')),
        TimeDistributed(MaxPooling1D(pool_size=1)),
        TimeDistributed(Flatten()),
        LSTM(128, return_sequences=False),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

def calculate_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    num_classes = cm.shape[0]

    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'sensitivity': 0,
        'specificity': 0
    }

    # For binary classification
    if num_classes == 2:
        tn, fp, fn, tp = cm.ravel()
        metrics['sensitivity'] = tp / (tp + fn)
        metrics['specificity'] = tn / (tn + fp)
    # For multi-class classification
    else:
        # Calculate sensitivity (recall for each class)
        recalls = recall_score(y_true, y_pred, average=None, zero_division=0)
        metrics['sensitivity'] = np.mean(recalls)

        # Calculate specificity (requires more complex calculation)
        specificities = []
        for i in range(num_classes):
            tn = np.sum(np.delete(np.delete(cm, i, axis=0), i, axis=1))
            fp = np.sum(cm[:, i]) - cm[i, i]
            specificities.append(tn / (tn + fp))
        metrics['specificity'] = np.mean(specificities)

    return metrics, cm

In [18]:
# Perform Leave-One-Subject-Out cross-validation
for fold, (train_idx, test_idx) in enumerate(logo.split(X, y, groups)):
    print(f"\n=== Fold {fold+1} - Testing on subject {groups[test_idx[0]]} ===")

    # Split data
    X_train, X_test = X_reshaped[train_idx], X_reshaped[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Get input shape
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])  # (timesteps, features, channels)
    num_classes = len(np.unique(y))

    # Train and evaluate Conv-LSTM
    print("\nTraining Conv-LSTM...")
    conv_lstm_model = create_conv_lstm(input_shape, num_classes)
    conv_lstm_history = conv_lstm_model.fit(X_train, y_train,
                                           epochs=20,
                                           batch_size=32,
                                           validation_split=0.2,
                                           verbose=1)

    # Evaluate Conv-LSTM
    conv_lstm_pred = np.argmax(conv_lstm_model.predict(X_test), axis=1)

    # Calculate metrics
    conv_lstm_metrics, conv_lstm_cm = calculate_metrics(y_test, conv_lstm_pred)

    # Store results
    results_conv_lstm['accuracy'].append(conv_lstm_metrics['accuracy'])
    results_conv_lstm['precision'].append(conv_lstm_metrics['precision'])
    results_conv_lstm['recall'].append(conv_lstm_metrics['recall'])
    results_conv_lstm['sensitivity'].append(conv_lstm_metrics['sensitivity'])
    results_conv_lstm['specificity'].append(conv_lstm_metrics['specificity'])

    # Print metrics
    print(f"\nConv-LSTM Metrics for fold {fold+1}:")
    print(f"Accuracy: {conv_lstm_metrics['accuracy']:.4f}")
    print(f"Precision: {conv_lstm_metrics['precision']:.4f}")
    print(f"Recall: {conv_lstm_metrics['recall']:.4f}")
    print(f"Sensitivity: {conv_lstm_metrics['sensitivity']:.4f}")
    print(f"Specificity: {conv_lstm_metrics['specificity']:.4f}")
    print("Confusion Matrix:")
    print(conv_lstm_cm)


=== Fold 1 - Testing on subject AP01 ===

Training Conv-LSTM...
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


175/175 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.8963 - loss: 0.6135 - val_accuracy: 0.7978 - val_loss: 1.3871
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9455 - loss: 0.2220 - val_accuracy: 0.7978 - val_loss: 1.1993
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9443 - loss: 0.2123 - val_accuracy: 0.7978 - val_loss: 1.1035
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9449 - loss: 0.2118 - val_accuracy: 0.7978 - val_loss: 1.1819
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9415 - loss: 0.2175 - val_accuracy: 0.7978 - val_loss: 1.3077
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9454 - loss: 0.2085 - val_accuracy: 0.7978 - val_loss: 1.0447
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9455 - loss: 0.2063 - val_accuracy: 0.7978 - val_loss: 1.1221
Epoch 8/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9484 - loss: 0.2024 - val_accuracy: 0.797

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9271 - loss: 0.5498 - val_accuracy: 0.7987 - val_loss: 1.0313
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9543 - loss: 0.1976 - val_accuracy: 0.7987 - val_loss: 0.8977
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9515 - loss: 0.1933 - val_accuracy: 0.7987 - val_loss: 0.8536
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9577 - loss: 0.1838 - val_accuracy: 0.7987 - val_loss: 0.9152
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.9546 - loss: 0.1858 - val_accuracy: 0.7987 - val_loss: 0.9708
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9536 - loss: 0.1778 - val_accuracy: 0.7987 - val_loss: 0.9203
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9540 - loss: 0.1871 - val_accuracy: 0.7987 - val_loss: 0.7987
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9519 - loss: 0.1822 - val_accuracy: 0.798

...

178/178 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.8989 - loss: 0.6056 - val_accuracy: 0.8000 - val_loss: 0.9482
Epoch 2/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9301 - loss: 0.2724 - val_accuracy: 0.8000 - val_loss: 0.8248
Epoch 3/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9281 - loss: 0.2677 - val_accuracy: 0.8000 - val_loss: 0.8519
Epoch 4/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9372 - loss: 0.2432 - val_accuracy: 0.8000 - val_loss: 0.8124
Epoch 5/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9337 - loss: 0.2444 - val_accuracy: 0.8000 - val_loss: 0.7970
Epoch 6/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9329 - loss: 0.2437 - val_accuracy: 0.8000 - val_loss: 0.7208
Epoch 7/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9318 - loss: 0.2446 - val_accuracy: 0.8000 - val_loss: 0.8788
Epoch 8/20
178/178 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.9327 - loss: 0.2460 - val_accuracy: 0.800

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


172/172 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.9186 - loss: 0.5496 - val_accuracy: 0.7968 - val_loss: 0.8765
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9569 - loss: 0.1885 - val_accuracy: 0.7968 - val_loss: 0.8022
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9581 - loss: 0.1875 - val_accuracy: 0.7968 - val_loss: 0.7800
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9529 - loss: 0.1822 - val_accuracy: 0.7968 - val_loss: 0.7700
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9579 - loss: 0.1735 - val_accuracy: 0.7968 - val_loss: 0.7649
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9527 - loss: 0.1994 - val_accuracy: 0.7968 - val_loss: 0.8012
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9570 - loss: 0.1779 - val_accuracy: 0.7968 - val_loss: 0.7596
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.9561 - loss: 0.1813 - val_accuracy: 0.796

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


181/181 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.9165 - loss: 0.5538 - val_accuracy: 0.8988 - val_loss: 0.3384
Epoch 2/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9591 - loss: 0.1821 - val_accuracy: 0.8988 - val_loss: 0.3205
Epoch 3/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9565 - loss: 0.1910 - val_accuracy: 0.8988 - val_loss: 0.3369
Epoch 4/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9547 - loss: 0.1831 - val_accuracy: 0.8988 - val_loss: 0.3220
Epoch 5/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.9543 - loss: 0.1827 - val_accuracy: 0.8988 - val_loss: 0.3401
Epoch 6/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9570 - loss: 0.1779 - val_accuracy: 0.8988 - val_loss: 0.3288
Epoch 7/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9562 - loss: 0.1818 - val_accuracy: 0.8988 - val_loss: 0.3334
Epoch 8/20
181/181 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.9590 - loss: 0.1731 - val_accuracy: 0.898

In [19]:
# Calculate and print aggregated results
def print_aggregated_results(results, model_name):
    print(f"\n=== Aggregated Results for {model_name} ===")
    for metric in results:
        values = results[metric]
        print(f"{metric.capitalize()}: Mean = {np.mean(values):.4f}, Std = {np.std(values):.4f}")

print_aggregated_results(results_conv_lstm, "Conv-LSTM")




=== Aggregated Results for Conv-LSTM ===
Accuracy: Mean = 0.9212, Std = 0.0575
Precision: Mean = 0.8519, Std = 0.1036
Recall: Mean = 0.9212, Std = 0.0575
Sensitivity: Mean = 0.2900, Std = 0.0554
Specificity: Mean = 0.7100, Std = 0.0554
